1.plot_one_mask_GlobalAndLocal (SERIES, figsize = (14, 14), alpha = 0.35)  
Basic visualization functions:  
In the whole image, a mask is visually highlighted, the background pixel value becomes smaller, and the contrast is increased.  
Take a mask area from the original image for local visualization, and the background is 0 pixels.  

Understand the data  
2.plot_one_ClassID (dataframe, ClassId, nums = 10, figsize = (14, 14), alpha = 0.35)  
Global and local visualization of different graphs of a certain ClassID. Take the first 10 pictures from the training set. The default arrangement is 10 rows and 3 columns. Each row is a different picture. The first column is the original image, the second column is the global mask, and the third column is the local area visualization. Some ClassIDs do not know what it is, you can understand it through this visualization.  

3.plot_one_Attribute (dataframe, Attribute, figsize = (14,14), alpha = 0.35)  
Global and local visualization of different ClassIDs of a certain Attribute. The default arrangement is n rows and 3 columns, each row is a picture of a different ClassId, the first column is the original image, the second column is the global mask, and the third column is the local area visualization. I want to understand the characteristics of attributes through this process.  
    

Some visualizations to understand properties such as length, vein, etc.    
Understand the relationship between level2 and level1   

TODU:  
Test truth value comparison:  
4.The visualization of two overlapping masks in one picture, used to compare the difference between prediction and truth mask.   

1.plot_one_mask_GlobalAndLocal(SERIES, figsize=(14, 14) ,alpha = 0.35)  
可视化基本功能：  
在全图中 可视化突出一个掩模，背景像素值变小，增加对比度。  
从原图中截取一个掩模的区域 进行局部可视化，背景为0像素。  

了解数据   
2.plot_one_ClassID(dataframe,ClassId,nums=10,figsize=(14, 14),alpha = 0.35 )   
某一类ClassID 不同图的全局和局部可视化。从训练集中取前面10个图片，排列方式默认是10行3列，每一行是不同的图片，第一列是原图，第二列是全局掩模，第三列是局部区域可视化。有些ClassID不知道是什么东西，可以通过这个可视化了解下。     

3.plot_one_Attribute(dataframe,Attribute,figsize = (14,14),alpha = 0.35)    
某一类Attribute的不同ClassID的全局和局部可视化。排列方式默认是n行3列，每一行是不同的ClassId的图片，第一列是原图，第二列是全局掩模，第三列是局部区域可视化。想通过这个过程了解属性有怎样的特征。  
    

一些可视化搞明白长度、脉络等属性    
搞明白level2和level1的关系  

TODU:  
测试真值对比：  
4.一张图片中 两个有重叠掩模的可视化，用于比较预测和真值掩模的差异。    


In [ ]:
import pandas as pd
import matplotlib.image as mpimg
import numpy as np
from matplotlib import pyplot as plt
import gc
import cv2

In [ ]:
train_df = pd.read_csv('/kaggle/input/imaterialist-fashion-2020-fgvc7/train.csv')

In [ ]:
train_df.head()

In [ ]:
def rle_to_mask(rle_string,height,width):
    # https://www.kaggle.com/tanreinama/prediction-and-submission-of-attributes
    rows, cols = height, width
    if rle_string == -1:
        return np.zeros((height, width))
    else:
        rleNumbers = [int(numstring) for numstring in rle_string.split(' ')]
        rlePairs = np.array(rleNumbers).reshape(-1,2)
        img = np.zeros(rows*cols,dtype=np.uint8)
        for index,length in rlePairs:
            index -= 1
            img[index:index+length] = 255
        img = img.reshape(cols,rows)
        img = img.T
        return img

# plot_one_mask_GlobalAndLocal

In [ ]:
def plot_one_mask_GlobalAndLocal(SERIES, figsize=(14, 14) ,alpha = 0.35):
    
    mask = rle_to_mask(SERIES['EncodedPixels'],SERIES['Height'],SERIES['Width'])
    image = cv2.imread("../input/imaterialist-fashion-2020-fgvc7/train/"+str(SERIES['ImageId'])+".jpg")
    b,g,r=cv2.split(image)
    image = cv2.merge([r,g,b])
    
    assert image.shape[0:2] == mask.shape[0:2]
    shape = image.shape[0:2]
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=figsize)
    ax[0].imshow(image)
    ax[0].set_title('ImageId: '+SERIES['ImageId'])
    
    ax[1].imshow(image)
    ax[1].imshow(mask, alpha=alpha) # 重叠 : overlapped
    ax[0].axis('off')
    ax[1].axis('off')
    ax[1].set_title('ClassId: '+str(SERIES['ClassId']))

    image[mask==0] = 255 # 背景为空白 : background is white
    where = np.where(image < 255) # 取掩模最小区域 : minimum mask area 
    if len(where[0]) > 0 and len(where[1]) > 0:
        y1,y2,x1,x2 = min(where[0]),max(where[0]),min(where[1]),max(where[1])
    ax[2].imshow(image[y1:y2,x1:x2])
    ax[2].set_title('AttributesIds: '+SERIES['AttributesIds'])
    
    ax[0].axis('off')
    ax[1].axis('off')    
    ax[2].axis('off')    
    plt.show()
    gc.collect()

# plot_one_ClassID(dataframe,ClassId,nums=10,figsize=(14, 14),alpha = 0.35 )

In [ ]:
def plot_one_ClassID(dataframe,ClassId,nums=10,figsize=(14, 14),alpha = 0.35 ):
    # find
    result = dataframe[dataframe.ClassId == ClassId][0:nums]    
    
    # plot
    for index,ser in result.iterrows():
        plot_one_mask_GlobalAndLocal(ser,figsize,alpha)    

In [ ]:
plot_one_ClassID(train_df , 1)

# plot_one_Attribute(dataframe,Attribute,figsize = (14,14),alpha = 0.35)

In [ ]:
def plot_one_Attribute(dataframe,Attribute,figsize = (14,14),alpha = 0.35):
    # find
    # 筛选出有某个属性的样本 : Filter out samples with a certain attribute
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html#pandas.Series.str.contains
    Attribute = str(Attribute)
    AttributesId_sample = dataframe[dataframe['AttributesIds'].str.contains(Attribute, regex=False,na= False)] # na用来把NaN变为False : Na is used to make Nan false
    
    # 根据ClassId进行分组，每组取一个样本 : Group according to ClassId, one sample for each group.    
    # https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html
    # as_index=False这样ClassId就不会被当成索引 : as_index = False so that ClassId will not be used as an index
    result = AttributesId_sample.groupby(['ClassId'],as_index=False).first() 

    # plot
    for index,ser in result.iterrows():
        plot_one_mask_GlobalAndLocal(ser,figsize,alpha)    


In [ ]:
plot_one_Attribute(train_df , 317)